In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark=SparkSession.builder.getOrCreate()

In [174]:
#reading csv file
df=spark.read.format("csv")\
.option("header","true")\
.option("inferSchema","true")\
.load("\data\retail-data\by-day\2010-12-01.csv")

# creating temporary view

In [175]:
df.createOrReplaceTempView("dftable")
spark.sql("select * from dftable").show(2)

+---------+---------+--------------------+--------+----------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|     InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+----------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|01-12-2010 08:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|01-12-2010 08:26|     3.39|     17850|United Kingdom|
+---------+---------+--------------------+--------+----------------+---------+----------+--------------+
only showing top 2 rows



In [116]:
df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)



# working with strings

# lit()

In [8]:
#Converting to Spark Types.lit function converts a type in another language to its correspnding Spark representation.
from pyspark.sql.functions import lit
df.select(lit(5), lit("five"), lit(5.0)).show(5)

+---+----+---+
|  5|five|5.0|
+---+----+---+
|  5|five|5.0|
|  5|five|5.0|
|  5|five|5.0|
|  5|five|5.0|
|  5|five|5.0|
+---+----+---+
only showing top 5 rows



In [12]:
#in sql
spark.sql("select 5,'five',5.0 from dftable").show(5)

+---+----+---+
|  5|five|5.0|
+---+----+---+
|  5|five|5.0|
|  5|five|5.0|
|  5|five|5.0|
|  5|five|5.0|
|  5|five|5.0|
+---+----+---+
only showing top 5 rows



# Working with Booleans

In [15]:
from pyspark.sql.functions import col
df.where("InvoiceNo != 536365")\
.select("InvoiceNo", "Description")\
.show(5)

+---------+--------------------+
|InvoiceNo|         Description|
+---------+--------------------+
|   536366|HAND WARMER UNION...|
|   536366|HAND WARMER RED P...|
|   536367|ASSORTED COLOUR B...|
|   536367|POPPY'S PLAYHOUSE...|
|   536367|POPPY'S PLAYHOUSE...|
+---------+--------------------+
only showing top 5 rows



In [6]:
#in sql
spark.sql("select InvoiceNo,Description from dftable where InvoiceNo!=536365").show(5)

+---------+--------------------+
|InvoiceNo|         Description|
+---------+--------------------+
|   536366|HAND WARMER UNION...|
|   536366|HAND WARMER RED P...|
|   536367|ASSORTED COLOUR B...|
|   536367|POPPY'S PLAYHOUSE...|
|   536367|POPPY'S PLAYHOUSE...|
+---------+--------------------+
only showing top 5 rows



In [21]:
#applying multiple filters
from pyspark.sql.functions import instr
priceFilter = col("UnitPrice") > 600
descripFilter = instr(df.Description, "POSTAGE") >= 1
df.where(col("StockCode").contains("DOT")).where(priceFilter | descripFilter).show()

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|   Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|   536544|      DOT|DOTCOM POSTAGE|       1|2010-12-01 14:32:00|   569.77|      null|United Kingdom|
|   536592|      DOT|DOTCOM POSTAGE|       1|2010-12-01 17:06:00|   607.49|      null|United Kingdom|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+



In [12]:
#applying multiple filters in single where clause
df.where("StockCode like '%DOT%' and (UnitPrice>600 or Description like '%POSTAGE%')").show(5)

+---------+---------+--------------+--------+----------------+---------+----------+--------------+
|InvoiceNo|StockCode|   Description|Quantity|     InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------+--------+----------------+---------+----------+--------------+
|   536544|      DOT|DOTCOM POSTAGE|       1|01-12-2010 14:32|   569.77|      null|United Kingdom|
|   536592|      DOT|DOTCOM POSTAGE|       1|01-12-2010 17:06|   607.49|      null|United Kingdom|
+---------+---------+--------------+--------+----------------+---------+----------+--------------+



In [13]:
#in sql
spark.sql("SELECT * FROM dfTable WHERE StockCode in ('DOT') AND (UnitPrice > 600 OR Description like '%POSTAGE%')").show()

+---------+---------+--------------+--------+----------------+---------+----------+--------------+
|InvoiceNo|StockCode|   Description|Quantity|     InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------+--------+----------------+---------+----------+--------------+
|   536544|      DOT|DOTCOM POSTAGE|       1|01-12-2010 14:32|   569.77|      null|United Kingdom|
|   536592|      DOT|DOTCOM POSTAGE|       1|01-12-2010 17:06|   607.49|      null|United Kingdom|
+---------+---------+--------------+--------+----------------+---------+----------+--------------+



In [10]:
DOTCodeFilter = col("StockCode") == "DOT"
priceFilter = col("UnitPrice") > 600
descripFilter = instr(col("Description"), "POSTAGE") >= 1
df.withColumn("isExpensive", DOTCodeFilter & (priceFilter | descripFilter))\
.where("isExpensive")\
.select("unitPrice", "isExpensive").show()

+---------+-----------+
|unitPrice|isExpensive|
+---------+-----------+
|   569.77|       true|
|   607.49|       true|
+---------+-----------+



In [24]:
#applying conditions single expression
df.withColumn("isExpensive", expr("StockCode =='DOT' and (UnitPrice > 600 or instr(Description, 'POSTAGE') >= 1)")).show(5)

+---------+---------+--------------------+--------+----------------+---------+----------+--------------+-----------+
|InvoiceNo|StockCode|         Description|Quantity|     InvoiceDate|UnitPrice|CustomerID|       Country|isExpensive|
+---------+---------+--------------------+--------+----------------+---------+----------+--------------+-----------+
|   536365|   85123A|WHITE HANGING HEA...|       6|01-12-2010 08:26|     2.55|     17850|United Kingdom|      false|
|   536365|    71053| WHITE METAL LANTERN|       6|01-12-2010 08:26|     3.39|     17850|United Kingdom|      false|
|   536365|   84406B|CREAM CUPID HEART...|       8|01-12-2010 08:26|     2.75|     17850|United Kingdom|      false|
|   536365|   84029G|KNITTED UNION FLA...|       6|01-12-2010 08:26|     3.39|     17850|United Kingdom|      false|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|01-12-2010 08:26|     3.39|     17850|United Kingdom|      false|
+---------+---------+--------------------+--------+-------------

In [5]:
#in sql
spark.sql("SELECT UnitPrice, (StockCode = 'DOT' AND (UnitPrice > 600 OR instr(Description, 'POSTAGE') >= 1)) as isExpensive FROM dfTable WHERE (StockCode = 'DOT' AND(UnitPrice > 600 OR instr(Description, 'POSTAGE') >= 1))").show()

+---------+-----------+
|UnitPrice|isExpensive|
+---------+-----------+
|   569.77|       true|
|   607.49|       true|
+---------+-----------+



# Working with numbers

In [22]:
from pyspark.sql.functions import expr, pow
fabricatedQuantity = pow(col("Quantity") * col("UnitPrice"), 2) + 5
df.select(expr("CustomerId"), fabricatedQuantity.alias("realQuantity")).show(2)

+----------+------------------+
|CustomerId|      realQuantity|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 2 rows



In [23]:
#in sql
spark.sql("select CustomerId,pow(Quantity*UnitPrice,2)+5 as realQuantity from dftable").show(2)

+----------+------------------+
|CustomerId|      realQuantity|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 2 rows



In [24]:
df.selectExpr(
"CustomerId",
"(POWER((Quantity * UnitPrice), 2.0) + 5) as realQuantity").show(2)

+----------+------------------+
|CustomerId|      realQuantity|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 2 rows



In [25]:
#in sql
spark.sql("select CustomerId,pow(Quantity*UnitPrice,2.0)+5 as realQuantity from dftable").show(2)

+----------+------------------+
|CustomerId|      realQuantity|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 2 rows



# round()

In [55]:
#rounding of
from pyspark.sql.functions import lit, round, bround
df.select(round(lit(2.55678999999),3).alias("round"), bround(lit(2.5)).alias("bround")).show(2)

+-----+------+
|round|bround|
+-----+------+
|2.557|   2.0|
|2.557|   2.0|
+-----+------+
only showing top 2 rows



In [56]:
df.selectExpr("round(UnitPrice, 1) as rounded_of", "UnitPrice").show(5)

+----------+---------+
|rounded_of|UnitPrice|
+----------+---------+
|       2.6|     2.55|
|       3.4|     3.39|
|       2.8|     2.75|
|       3.4|     3.39|
|       3.4|     3.39|
+----------+---------+
only showing top 5 rows



In [40]:
#round and bround does similiar functions like ceil and floor respectively
spark.sql("select round(2.5),ceil(2.5),bround(2.5),floor(2.5) from dftable").show(2)

+-------------+---------+--------------+----------+
|round(2.5, 0)|CEIL(2.5)|bround(2.5, 0)|FLOOR(2.5)|
+-------------+---------+--------------+----------+
|            3|        3|             2|         2|
|            3|        3|             2|         2|
+-------------+---------+--------------+----------+
only showing top 2 rows



In [41]:
#to know about statistics of each columns use describe
df.describe().show()

+-------+-----------------+------------------+--------------------+------------------+----------------+------------------+------------------+--------------+
|summary|        InvoiceNo|         StockCode|         Description|          Quantity|     InvoiceDate|         UnitPrice|        CustomerID|       Country|
+-------+-----------------+------------------+--------------------+------------------+----------------+------------------+------------------+--------------+
|  count|             3108|              3108|                3098|              3108|            3108|              3108|              1968|          3108|
|   mean| 536516.684944841|27834.304044117645|                null| 8.627413127413128|            null| 4.151946589446603|15661.388719512195|          null|
| stddev|72.89447869788873|17407.897548583845|                null|26.371821677029203|            null|15.638659854603892|1854.4496996893627|          null|
|    min|           536365|             10002| 4 PURPLE FL

# monotonically_increasing_id()

In [28]:
from pyspark.sql.functions import monotonically_increasing_id
df.selectExpr("monotonically_increasing_id()+1 as row_id").show(2)

+------+
|row_id|
+------+
|     1|
|     2|
+------+
only showing top 2 rows



In [38]:
spark.sql("select row_number() over(order by 1) as id from dfTable").show(5)

+---+
| id|
+---+
|  1|
|  2|
|  3|
|  4|
|  5|
+---+
only showing top 5 rows



# Working with strings

# 1) initcap

In [43]:
#capitalize every first word
from pyspark.sql.functions import initcap
df.select(initcap("Description").alias("Description")).show(5,False)

+-----------------------------------+
|Description                        |
+-----------------------------------+
|White Hanging Heart T-light Holder |
|White Metal Lantern                |
|Cream Cupid Hearts Coat Hanger     |
|Knitted Union Flag Hot Water Bottle|
|Red Woolly Hottie White Heart.     |
+-----------------------------------+
only showing top 5 rows



In [45]:
#in sql
spark.sql("SELECT initcap(Description) FROM dfTable").show(5,False)

+-----------------------------------+
|initcap(Description)               |
+-----------------------------------+
|White Hanging Heart T-light Holder |
|White Metal Lantern                |
|Cream Cupid Hearts Coat Hanger     |
|Knitted Union Flag Hot Water Bottle|
|Red Woolly Hottie White Heart.     |
+-----------------------------------+
only showing top 5 rows



# 2) lower, upper

In [46]:
#converting to uppercase and lowercase
from pyspark.sql.functions import lower, upper
df.selectExpr("Description","lower(Description)","upper(Description)").show(2)

+--------------------+--------------------+--------------------+
|         Description|  lower(Description)|  upper(Description)|
+--------------------+--------------------+--------------------+
|WHITE HANGING HEA...|white hanging hea...|WHITE HANGING HEA...|
| WHITE METAL LANTERN| white metal lantern| WHITE METAL LANTERN|
+--------------------+--------------------+--------------------+
only showing top 2 rows



# 3) concat

In [37]:
df.selectExpr("upper(Description)","concat(Description,',',Stockcode)").show(5,False)

+-----------------------------------+------------------------------------------+
|upper(Description)                 |concat(Description, ,, Stockcode)         |
+-----------------------------------+------------------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER |WHITE HANGING HEART T-LIGHT HOLDER,85123A |
|WHITE METAL LANTERN                |WHITE METAL LANTERN,71053                 |
|CREAM CUPID HEARTS COAT HANGER     |CREAM CUPID HEARTS COAT HANGER,84406B     |
|KNITTED UNION FLAG HOT WATER BOTTLE|KNITTED UNION FLAG HOT WATER BOTTLE,84029G|
|RED WOOLLY HOTTIE WHITE HEART.     |RED WOOLLY HOTTIE WHITE HEART.,84029E     |
+-----------------------------------+------------------------------------------+
only showing top 5 rows



# 4) trim, pad

In [27]:
#adding or removing spaces around a string
from pyspark.sql.functions import lit, ltrim, rtrim, rpad, lpad, trim
df.select(
ltrim(lit("   HELLO ")).alias("ltrim"),
rtrim(lit(" HELLO ")).alias("rtrim"),
trim(lit(" HELLO ")).alias("trim"),
lpad(lit("HELLO"), 8, " ").alias("lp"),
rpad(lit("HELLO"),8, " ").alias("rp")).show(2)

+------+------+-----+--------+--------+
| ltrim| rtrim| trim|      lp|      rp|
+------+------+-----+--------+--------+
|HELLO | HELLO|HELLO|   HELLO|HELLO   |
|HELLO | HELLO|HELLO|   HELLO|HELLO   |
+------+------+-----+--------+--------+
only showing top 2 rows



In [59]:
spark.sql("SELECT ltrim(' HELLLOOOO ') as ltrim,rtrim(' HELLLOOOO ') as rtrim,trim(' HELLLOOOO ') as trim,lpad('HELLOOOO ', 3, ' ') as lpad,\
rpad('HELLOOOO ', 10, ' ') as rpad FROM dfTable").show(2)

+----------+----------+---------+----+----------+
|     ltrim|     rtrim|     trim|lpad|      rpad|
+----------+----------+---------+----+----------+
|HELLLOOOO | HELLLOOOO|HELLLOOOO| HEL|HELLOOOO  |
|HELLLOOOO | HELLLOOOO|HELLLOOOO| HEL|HELLOOOO  |
+----------+----------+---------+----+----------+
only showing top 2 rows



# Working with Regular expressions

# 1) regexp_replace

In [63]:
#repacing the values
#syntax regexp_replace(column,regex_string_to_replace,replacement_string) 
from pyspark.sql.functions import regexp_replace
df.selectExpr("regexp_replace(Description,'BLACK|WHITE|RED|GREEN|BLUE','COLOR') as color_clean","Description").show(2)

+--------------------+--------------------+
|         color_clean|         Description|
+--------------------+--------------------+
|COLOR HANGING HEA...|WHITE HANGING HEA...|
| COLOR METAL LANTERN| WHITE METAL LANTERN|
+--------------------+--------------------+
only showing top 2 rows



In [67]:
df.selectExpr("InvoiceDate","trim(regexp_replace(InvoiceDate,'([0-9]+)(-){1}','')) as reg_exp").show(5,False)

+----------------+----------+
|InvoiceDate     |reg_exp   |
+----------------+----------+
|01-12-2010 08:26|2010 08:26|
|01-12-2010 08:26|2010 08:26|
|01-12-2010 08:26|2010 08:26|
|01-12-2010 08:26|2010 08:26|
|01-12-2010 08:26|2010 08:26|
+----------------+----------+
only showing top 5 rows



# 2)regexp_extract

In [89]:
#pulling out the first mentioned color
#syntax regexp_extract(column,regex_string_to_replace,capture_group) 
#0 as capture group means it will extract all the characters in string
#If the regex did not match, or the specified group did not match, an empty string is returned
from pyspark.sql.functions import regexp_extract
df.selectExpr("regexp_extract(Description,'(BLACK|WHITE|RED|GREEN|BLUE)',1) as color_clean","Description").show(5,False)

+-----------+-----------------------------------+
|color_clean|Description                        |
+-----------+-----------------------------------+
|WHITE      |WHITE HANGING HEART T-LIGHT HOLDER |
|WHITE      |WHITE METAL LANTERN                |
|           |CREAM CUPID HEARTS COAT HANGER     |
|           |KNITTED UNION FLAG HOT WATER BOTTLE|
|RED        |RED WOOLLY HOTTIE WHITE HEART.     |
+-----------+-----------------------------------+
only showing top 5 rows



In [73]:
from pyspark.sql.functions import regexp_extract
df.selectExpr("InvoiceDate","trim(regexp_extract(InvoiceDate,'([0-9]+)(-)([0-9]+)(-)([0-9]+)',3)) as reg_exp").show(5,False)

+----------------+-------+
|InvoiceDate     |reg_exp|
+----------------+-------+
|01-12-2010 08:26|12     |
|01-12-2010 08:26|12     |
|01-12-2010 08:26|12     |
|01-12-2010 08:26|12     |
|01-12-2010 08:26|12     |
+----------------+-------+
only showing top 5 rows



In [74]:
#in sql
spark.sql("SELECT regexp_replace(Description, 'BLACK|WHITE|RED|GREEN|BLUE', 'COLOR') as color_clean, \
          Description FROM dfTable").show(2)

+--------------------+--------------------+
|         color_clean|         Description|
+--------------------+--------------------+
|COLOR HANGING HEA...|WHITE HANGING HEA...|
| COLOR METAL LANTERN| WHITE METAL LANTERN|
+--------------------+--------------------+
only showing top 2 rows



# 3)translate

In [29]:
#replace all instances of a character with the indexed character in the replacement string
from pyspark.sql.functions import translate
df.select(translate(col("Description"), "LEET", "1337"),col("Description"))\
.show(5,False)

+-----------------------------------+-----------------------------------+
|translate(Description, LEET, 1337) |Description                        |
+-----------------------------------+-----------------------------------+
|WHI73 HANGING H3AR7 7-1IGH7 HO1D3R |WHITE HANGING HEART T-LIGHT HOLDER |
|WHI73 M37A1 1AN73RN                |WHITE METAL LANTERN                |
|CR3AM CUPID H3AR7S COA7 HANG3R     |CREAM CUPID HEARTS COAT HANGER     |
|KNI773D UNION F1AG HO7 WA73R BO7713|KNITTED UNION FLAG HOT WATER BOTTLE|
|R3D WOO11Y HO77I3 WHI73 H3AR7.     |RED WOOLLY HOTTIE WHITE HEART.     |
+-----------------------------------+-----------------------------------+
only showing top 5 rows



In [44]:
#in sql
spark.sql("SELECT translate(Description, 'LEET', '1337'), Description FROM dfTable").show(2)

+----------------------------------+--------------------+
|translate(Description, LEET, 1337)|         Description|
+----------------------------------+--------------------+
|              WHI73 HANGING H3A...|WHITE HANGING HEA...|
|               WHI73 M37A1 1AN73RN| WHITE METAL LANTERN|
+----------------------------------+--------------------+
only showing top 2 rows



In [48]:
#in sql
spark.sql("SELECT Description FROM dfTable \
WHERE instr(Description, 'BLACK') >= 1 OR instr(Description, 'WHITE') >= 1").show(2)

+--------------------+
|         Description|
+--------------------+
|WHITE HANGING HEA...|
| WHITE METAL LANTERN|
+--------------------+
only showing top 2 rows



# working with dates and times

# 1) current_date,current_timestamp

In [94]:
from pyspark.sql.functions import current_date,current_timestamp,col
dateDF = spark.range(10)\
.withColumn("today", current_date())\
.withColumn("now", current_timestamp())
dateDF.createOrReplaceTempView("dateTable")
dateDF.show(5,False)

+---+----------+----------------------+
|id |today     |now                   |
+---+----------+----------------------+
|0  |2022-02-26|2022-02-26 20:08:19.54|
|1  |2022-02-26|2022-02-26 20:08:19.54|
|2  |2022-02-26|2022-02-26 20:08:19.54|
|3  |2022-02-26|2022-02-26 20:08:19.54|
|4  |2022-02-26|2022-02-26 20:08:19.54|
+---+----------+----------------------+
only showing top 5 rows



# 2) date_add,date_sub

In [95]:
#let’s add and subtract five days from today
from pyspark.sql.functions import date_add, date_sub
dateDF.selectExpr("date_sub(today, 5) as date_subtrction","date_add(today, 5) as date_addition").show(1)

+---------------+-------------+
|date_subtrction|date_addition|
+---------------+-------------+
|     2022-02-21|   2022-03-03|
+---------------+-------------+
only showing top 1 row



# 3) datediff

In [98]:
#datediff function that will return the number of days in between two dates
from pyspark.sql.functions import datediff, months_between, to_date,lit
dateDF.withColumn("week_ago",expr("date_sub(today, 7)"))\
.selectExpr("datediff(today,week_ago) as date_difference").show(1)

+---------------+
|date_difference|
+---------------+
|              7|
+---------------+
only showing top 1 row



# 4) to_date

In [101]:
#months_between gives you the number of months between two dates
#The to_date function allows you to convert a string to a date
dateDF.selectExpr("to_date('2016-01-01') as start",
              "to_date('2017-05-22') as end").show(1)

+----------+----------+
|     start|       end|
+----------+----------+
|2016-01-01|2017-05-22|
+----------+----------+
only showing top 1 row



# 5) month_between

In [108]:
dateDF.selectExpr("to_date('2016-01-01') as start","to_date('2017-05-22') as end") \
.selectExpr("ceil(months_between(end,start)) as month_difference").show(1)

+----------------+
|month_difference|
+----------------+
|              17|
+----------------+
only showing top 1 row



In [119]:
#in sql
spark.sql("SELECT to_date('2016-01-01') as date, months_between('2017-01-01','2016-01-01') as mdiff,\
          datediff('2017-01-01','2016-01-01') as ddiff FROM dateTable").show(2)

+----------+-----+-----+
|      date|mdiff|ddiff|
+----------+-----+-----+
|2016-01-01| 12.0|  366|
|2016-01-01| 12.0|  366|
+----------+-----+-----+
only showing top 2 rows



In [121]:
#Spark will not throw an error if it cannot parse the date; rather, it will just return null
#To illustrate, let’s take a look at the date format that has switched from year-month-day to 
#year-day-month.Spark will fail to parse this date and silently return null instead
dateDF.select(to_date(lit("2016-20-12")).alias("date1"),to_date(lit("2017-12-11")).alias("date2")).show(2)

+-----+----------+
|date1|     date2|
+-----+----------+
| null|2017-12-11|
| null|2017-12-11|
+-----+----------+
only showing top 2 rows



In [124]:
#way to avoid these issues entirely
#The first step is to remember that we need to specify our date format according to the JavaSimpleDateFormat standard.
dateFormat = "yyyy-dd-MM"
cleanDateDF = spark.range(1).select(
to_date(lit("2017-27-11"), dateFormat).alias("date"),
to_date(lit("2017-20-12"), dateFormat).alias("date2"))
cleanDateDF.show()
cleanDateDF.createOrReplaceTempView("dateTable2")

+----------+----------+
|      date|     date2|
+----------+----------+
|2017-11-27|2017-12-20|
+----------+----------+



# 6) from_unixtime, unix_timestamp

In [125]:
#in sql
spark.sql("SELECT from_unixtime(unix_timestamp('27-10-2021', 'dd-MM-yyyy'),'MM/dd/yyyy') as date, to_date(date2, 'yyyy-dd-MM') as date2 FROM dateTable2").show()

+----------+----------+
|      date|     date2|
+----------+----------+
|10/27/2021|2017-12-20|
+----------+----------+



# 7) to_timestamp

In [59]:
#to_timestamp, which always requires a format to be specified:
from pyspark.sql.functions import to_timestamp
cleanDateDF.select(to_timestamp(col("date"), dateFormat)).show()

+----------------------------------+
|to_timestamp(`date`, 'yyyy-dd-MM')|
+----------------------------------+
|               2017-11-12 00:00:00|
+----------------------------------+



In [60]:
#in sql
spark.sql("select to_timestamp(date,'yyyy-mm-dd') from dateTable2").show()

+---------------------------------------------+
|to_timestamp(datetable2.`date`, 'yyyy-mm-dd')|
+---------------------------------------------+
|                          2017-11-12 00:00:00|
+---------------------------------------------+



In [78]:
#Casting between dates and timestamps 
#in sql
spark.sql("SELECT cast(to_date('2017-01-01', 'yyyy-MM-dd') as timestamp)").show()

+------------------------------------------------------+
|CAST(to_date('2017-01-01', 'yyyy-MM-dd') AS TIMESTAMP)|
+------------------------------------------------------+
|                                   2017-01-01 00:00:00|
+------------------------------------------------------+



In [129]:
#comparing dates
cleanDateDF.where("date2 >'2017-12-12'").show()

+----------+----------+
|      date|     date2|
+----------+----------+
|2017-11-27|2017-12-20|
+----------+----------+



# working with null values

# 1) coalesce()

In [ ]:
#select the first non-null value from a set of columns by using the coalesce function.
from pyspark.sql.functions import coalesce
df.select(coalesce(col("Description"), col("CustomerId"))).show(5,False)

# 2) ifnull(), nullIf(), nvl(), nvl2()

In [131]:
#ifnull, nullIf, nvl, and nvl2
spark.sql("SELECT ifnull(null, 'return_value') as ifnull,nullif('value', 'value') as nullif,nvl(null, 'return_value') as nvl,\
          nvl2('not_null', 'return_value', 'else_value') as nvl2 FROM dfTable LIMIT 1").show()

+------------+------+------------+------------+
|      ifnull|nullif|         nvl|        nvl2|
+------------+------+------------+------------+
|return_value|  null|return_value|return_value|
+------------+------+------------+------------+



# 3) drop()

In [135]:
df.count()

3108

In [133]:
#drop, which removes rows that contain nulls.
#see the difference between the count values after using drop()
df.na.drop().count()

1968

In [136]:
#In SQL, we have to do this column by column:
spark.sql("SELECT * FROM dfTable WHERE Description IS NOT NULL").show(2)

+---------+---------+--------------------+--------+----------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|     InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+----------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|01-12-2010 08:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|01-12-2010 08:26|     3.39|     17850|United Kingdom|
+---------+---------+--------------------+--------+----------------+---------+----------+--------------+
only showing top 2 rows



In [67]:
#Specifying "any" as an argument drops a row if any of the values are null. Using “all” drops the
#row only if all values are null or NaN for that row:
df.na.drop("all")

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: double, Country: string]

In [68]:
#can also apply this to certain sets of columns by passing in an array of columns
df.na.drop("all", subset=["StockCode", "InvoiceNo"])

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: double, Country: string]

# 4) fill()

In [69]:
#Using the fill function, you can fill one or more columns with a set of values
#tofill null values of type string
df.na.fill("All Null values become this string")

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: double, Country: string]

In [70]:
#to specify null values to specific columns
df.na.fill("all", subset=["StockCode", "InvoiceNo"])

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: double, Country: string]

In [137]:
#specifying the fill values of each columns using dictionary
fill_cols_vals = {"StockCode": 5, "Description" : "No Value"}
df.na.fill(fill_cols_vals)

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: int, Country: string]

# 5) replace()

In [81]:
#replacing 
df.na.replace([""], ["UNKNOWN"], "Description")

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: double, Country: string]

# Working with Complex Types

# 1) struct

In [140]:
#structs : DataFrames within DataFrames
from pyspark.sql.functions import struct
df.selectExpr("struct(Description, InvoiceNo) as complex", "*").show(5,False)

+---------------------------------------------+---------+---------+-----------------------------------+--------+----------------+---------+----------+--------------+
|complex                                      |InvoiceNo|StockCode|Description                        |Quantity|InvoiceDate     |UnitPrice|CustomerID|Country       |
+---------------------------------------------+---------+---------+-----------------------------------+--------+----------------+---------+----------+--------------+
|[WHITE HANGING HEART T-LIGHT HOLDER, 536365] |536365   |85123A   |WHITE HANGING HEART T-LIGHT HOLDER |6       |01-12-2010 08:26|2.55     |17850     |United Kingdom|
|[WHITE METAL LANTERN, 536365]                |536365   |71053    |WHITE METAL LANTERN                |6       |01-12-2010 08:26|3.39     |17850     |United Kingdom|
|[CREAM CUPID HEARTS COAT HANGER, 536365]     |536365   |84406B   |CREAM CUPID HEARTS COAT HANGER     |8       |01-12-2010 08:26|2.75     |17850     |United Kingdom|
|[KN

In [141]:
complexDF = df.selectExpr("struct(Description,InvoiceNo) as complex")
complexDF.createOrReplaceTempView("complexDF")

In [143]:
#We now have a DataFrame with a column complex. We can query it just as we might another
#DataFrame, the only difference is that we use a dot syntax to do so, or the column method getField
complexDF.select(col("complex.Description")).show(2,False)

+----------------------------------+
|Description                       |
+----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER|
|WHITE METAL LANTERN               |
+----------------------------------+
only showing top 2 rows



In [144]:
#other method
complexDF.select(col("complex").getField("Description")).show(5)

+--------------------+
| complex.Description|
+--------------------+
|WHITE HANGING HEA...|
| WHITE METAL LANTERN|
|CREAM CUPID HEART...|
|KNITTED UNION FLA...|
|RED WOOLLY HOTTIE...|
+--------------------+
only showing top 5 rows



In [145]:
#We can also query all values in the struct by using *
complexDF.select("complex.*").show(5)

+--------------------+---------+
|         Description|InvoiceNo|
+--------------------+---------+
|WHITE HANGING HEA...|   536365|
| WHITE METAL LANTERN|   536365|
|CREAM CUPID HEART...|   536365|
|KNITTED UNION FLA...|   536365|
|RED WOOLLY HOTTIE...|   536365|
+--------------------+---------+
only showing top 5 rows



In [96]:
#in sql
spark.sql("SELECT complex.* FROM complexDF").show(5)

+--------------------+---------+
|         Description|InvoiceNo|
+--------------------+---------+
|WHITE HANGING HEA...|   536365|
| WHITE METAL LANTERN|   536365|
|CREAM CUPID HEART...|   536365|
|KNITTED UNION FLA...|   536365|
|RED WOOLLY HOTTIE...|   536365|
+--------------------+---------+
only showing top 5 rows



# 2) array

In [146]:
#arrays: take every single word in our Description column using split and convert that into a row in our DataFrame
from pyspark.sql.functions import split
df.select(split(col("Description"), " ")).show(2)

+-------------------------+
|split(Description,  , -1)|
+-------------------------+
|     [WHITE, HANGING, ...|
|     [WHITE, METAL, LA...|
+-------------------------+
only showing top 2 rows



In [100]:
#can access elements using index
df.select(split(col("Description"), " ").alias("array_col"))\
.selectExpr("array_col[0]","array_col[1]").show(5)

+------------+------------+
|array_col[0]|array_col[1]|
+------------+------------+
|       WHITE|     HANGING|
|       WHITE|       METAL|
|       CREAM|       CUPID|
|     KNITTED|       UNION|
|         RED|      WOOLLY|
+------------+------------+
only showing top 5 rows



In [105]:
#in sql
spark.sql("select split(Description,' ') as splitcol from dfTable").show(5)

+--------------------+
|            splitcol|
+--------------------+
|[WHITE, HANGING, ...|
|[WHITE, METAL, LA...|
|[CREAM, CUPID, HE...|
|[KNITTED, UNION, ...|
|[RED, WOOLLY, HOT...|
+--------------------+
only showing top 5 rows



In [147]:
#use size() to find the array Length
from pyspark.sql.functions import size
df.select(size(split("Description", ' '))).show(2)

+-------------------------------+
|size(split(Description,  , -1))|
+-------------------------------+
|                              5|
|                              3|
+-------------------------------+
only showing top 2 rows



In [149]:
#array_contains : We can also see whether this array contains a value
from pyspark.sql.functions import array_contains
df.selectExpr("array_contains(split(Description,' '),'WHITE') as array_contains").show(2)

+--------------+
|array_contains|
+--------------+
|          true|
|          true|
+--------------+
only showing top 2 rows



In [150]:
#in sql
spark.sql("SELECT array_contains(split(Description, ' '), 'WHITE') FROM dfTable").show(2)

+------------------------------------------------+
|array_contains(split(Description,  , -1), WHITE)|
+------------------------------------------------+
|                                            true|
|                                            true|
+------------------------------------------------+
only showing top 2 rows



# 3) explode()

In [152]:
#explode:The explode function takes a column that consists of arrays and creates one row per value in the array
from pyspark.sql.functions import split, explode
df.withColumn("splitted",split(col("Description"), " "))\
.withColumn("exploded", explode(col("splitted")))\
.select("Description", "InvoiceNo", "exploded").show(5,False)

+----------------------------------+---------+--------+
|Description                       |InvoiceNo|exploded|
+----------------------------------+---------+--------+
|WHITE HANGING HEART T-LIGHT HOLDER|536365   |WHITE   |
|WHITE HANGING HEART T-LIGHT HOLDER|536365   |HANGING |
|WHITE HANGING HEART T-LIGHT HOLDER|536365   |HEART   |
|WHITE HANGING HEART T-LIGHT HOLDER|536365   |T-LIGHT |
|WHITE HANGING HEART T-LIGHT HOLDER|536365   |HOLDER  |
+----------------------------------+---------+--------+
only showing top 5 rows



In [154]:
#in sql
spark.sql("select Description,InvoiceNo,explode(split(description,' ')) as exploded from dfTable").show(5)

+--------------------+---------+--------+
|         Description|InvoiceNo|exploded|
+--------------------+---------+--------+
|WHITE HANGING HEA...|   536365|   WHITE|
|WHITE HANGING HEA...|   536365| HANGING|
|WHITE HANGING HEA...|   536365|   HEART|
|WHITE HANGING HEA...|   536365| T-LIGHT|
|WHITE HANGING HEA...|   536365|  HOLDER|
+--------------------+---------+--------+
only showing top 5 rows



# 4) posexplode()

In [165]:
#similiar to explode but returns another columns which specifies the position for each array
df.selectExpr("posexplode(split(Description,' '))","Description").show(5,False)

+---+-------+----------------------------------+
|pos|col    |Description                       |
+---+-------+----------------------------------+
|0  |WHITE  |WHITE HANGING HEART T-LIGHT HOLDER|
|1  |HANGING|WHITE HANGING HEART T-LIGHT HOLDER|
|2  |HEART  |WHITE HANGING HEART T-LIGHT HOLDER|
|3  |T-LIGHT|WHITE HANGING HEART T-LIGHT HOLDER|
|4  |HOLDER |WHITE HANGING HEART T-LIGHT HOLDER|
+---+-------+----------------------------------+
only showing top 5 rows



# 5) create_map()

In [161]:
#maps: Maps are created by using the map function and key-value pairs of columns
from pyspark.sql.functions import create_map
df.select(create_map("InvoiceNo","Description").alias("complex_map")).show(5,False)

+-----------------------------------------------+
|complex_map                                    |
+-----------------------------------------------+
|[536365 -> WHITE HANGING HEART T-LIGHT HOLDER] |
|[536365 -> WHITE METAL LANTERN]                |
|[536365 -> CREAM CUPID HEARTS COAT HANGER]     |
|[536365 -> KNITTED UNION FLAG HOT WATER BOTTLE]|
|[536365 -> RED WOOLLY HOTTIE WHITE HEART.]     |
+-----------------------------------------------+
only showing top 5 rows



In [38]:
#You can query them by using the proper key:check each row for "WHITE METAL LANTERN" in the data and return its invoice 
df.select(create_map("Description","InvoiceNo").alias("complex_map"))\
.selectExpr("complex_map['WHITE METAL LANTERN']").show(2)

+--------------------------------+
|complex_map[WHITE METAL LANTERN]|
+--------------------------------+
|                            null|
|                          536365|
+--------------------------------+
only showing top 2 rows



In [160]:
#in sql
spark.sql("select map(InvoiceNo,Description) as complex_map from dfTable").show(5,False)

+-----------------------------------------------+
|complex_map                                    |
+-----------------------------------------------+
|[536365 -> WHITE HANGING HEART T-LIGHT HOLDER] |
|[536365 -> WHITE METAL LANTERN]                |
|[536365 -> CREAM CUPID HEARTS COAT HANGER]     |
|[536365 -> KNITTED UNION FLAG HOT WATER BOTTLE]|
|[536365 -> RED WOOLLY HOTTIE WHITE HEART.]     |
+-----------------------------------------------+
only showing top 5 rows



In [162]:
#You can also explode map types
df.select(create_map(col("Description"), col("InvoiceNo")).alias("complex_map"))\
.selectExpr("explode(complex_map)").show(5,False)

+-----------------------------------+------+
|key                                |value |
+-----------------------------------+------+
|WHITE HANGING HEART T-LIGHT HOLDER |536365|
|WHITE METAL LANTERN                |536365|
|CREAM CUPID HEARTS COAT HANGER     |536365|
|KNITTED UNION FLAG HOT WATER BOTTLE|536365|
|RED WOOLLY HOTTIE WHITE HEART.     |536365|
+-----------------------------------+------+
only showing top 5 rows



# working with json

# 1)to_json

In [30]:
#You can turn a StructType into a JSON string by using the to_json function
from pyspark.sql.functions import *
df1=df.selectExpr("to_json(struct(InvoiceNo,Description)) as json_string")
df1.show(5,False)

+--------------------------------------------------------------------------+
|json_string                                                               |
+--------------------------------------------------------------------------+
|{"InvoiceNo":"536365","Description":"WHITE HANGING HEART T-LIGHT HOLDER"} |
|{"InvoiceNo":"536365","Description":"WHITE METAL LANTERN"}                |
|{"InvoiceNo":"536365","Description":"CREAM CUPID HEARTS COAT HANGER"}     |
|{"InvoiceNo":"536365","Description":"KNITTED UNION FLAG HOT WATER BOTTLE"}|
|{"InvoiceNo":"536365","Description":"RED WOOLLY HOTTIE WHITE HEART."}     |
+--------------------------------------------------------------------------+
only showing top 5 rows



# 2) from_json

In [45]:
parseSchema = StructType((
StructField("InvoiceNo",StringType(),True),
StructField("Description",StringType(),True)))

In [56]:
#You can use the from_json function to parse the JSON data back in. This naturally requires you to 
#specify a schema
dfp1=df.selectExpr("to_json(struct(InvoiceNo,Description)) as json_string")
dfp1.select(from_json("json_string",parseSchema)).show(5,False)

+---------------------------------------------+
|from_json(json_string)                       |
+---------------------------------------------+
|[536365, WHITE HANGING HEART T-LIGHT HOLDER] |
|[536365, WHITE METAL LANTERN]                |
|[536365, CREAM CUPID HEARTS COAT HANGER]     |
|[536365, KNITTED UNION FLAG HOT WATER BOTTLE]|
|[536365, RED WOOLLY HOTTIE WHITE HEART.]     |
+---------------------------------------------+
only showing top 5 rows



# user defined functions

In [167]:
#user defined functions
udfExampleDF = spark.range(5).toDF("num")
def power3(double_value):
    return double_value ** 3

In [170]:
udfExampleDF.createOrReplaceTempView("udf_df")

In [168]:
#need to register the function to make it available as a DataFrame function
from pyspark.sql.functions import udf
power3udf = udf(power3,IntegerType())

In [169]:
from pyspark.sql.functions import col
udfExampleDF.select(power3udf("num").alias("cube_num")).show()

+--------+
|cube_num|
+--------+
|       0|
|       1|
|       8|
|      27|
|      64|
+--------+



In [171]:
#registering udf to work with sql
from pyspark.sql.types import IntegerType, DoubleType
spark.udf.register("power3py", power3, IntegerType())

<function __main__.power3(double_value)>

In [173]:
spark.sql("select power3py(num) as cube_num from udf_df").show()

+--------+
|cube_num|
+--------+
|       0|
|       1|
|       8|
|      27|
|      64|
+--------+

